# Entrega 2, Grupo 02 - Aprendizaje Bayesiano

- Santiago Alaniz,  5082647-6, santiago.alaniz@fing.edu.uy
- Bruno De Simone,  4914555-0, bruno.de.simone@fing.edu.uy
- María Usuca,      4891124-3, maria.usuca@fing.edu.uy

## Objetivo

Implementar un algoritmo de predicción de palabras utilizando Naive Bayes. El modelo se entrenará con datos de conversaciones de WhatsApp y se probará en un simulador de cliente. 

El algoritmo debe ser capaz de recomendar palabras basadas en las últimas N palabras ingresadas en una frase, donde N es un hiperparámetro que se variará para evaluar el desempeño del modelo. Además, el modelo se reentrenará al finalizar cada frase para adaptarse a nueva evidencia. 

La implementación debe ser eficiente en términos de uso de CPU, utilizando las estructuras de datos más adecuadas en Python.


## Diseño

### Carga del dump de conversaciones.

En la letra del laboratorio se menciona que se puede acceder al dump en formato `.csv`. Enrealidad, *WhatsApp* exporta las conversaciones en formato `.txt`. Detectamos el siguiente patron en el archivo de texto

```
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
    ***
  [dd/mm/aaaa hh:mm:ss] <nombre>: <mensaje>
```

Nos valemos del modulo `src.regex` para definir la expresion regular que detecta el patron y extrae los mensajes (`LOG_ENTRY_PATTERN`).

`LOG_ENTRY_PATTERN = f'{metadata_pattern}{message_pattern}(?=\n{metadata_pattern}|$)':`

Está diseñado para capturar una entrada completa del registro de chat, que incluye tanto los metadatos como el mensaje.aplicamos nociones de búsqueda hacia adelante que indica el final de una entrada.



In [ ]:
import re
from src.regex import LOG_ENTRY_PATTERN

FILE_PATH = './assets/chat.txt'
PATTERN = LOG_ENTRY_PATTERN

with open(FILE_PATH, 'r', encoding='utf-8') as file:
  chat = file.read()

matches = re.findall(PATTERN, chat)
messages = [ re.sub(r'\n', ' ', match[1]) for match in matches ]

messages[:10]

### Preprocesamiento de datos
- Decisiones sobre tratamiento de datos numéricos, faltantes, etc. antes de la aplicación de el algoritmo
- Selección/generación de atributos




**Carga de datos y Transformación**: A continuación vamos a realizar la carga de una conversación y su transformación para:
- Obtener solo las palabras en el diccionario.
- Pasar todas las palabras a minusculas.
- Armar una lista y ordenar

In [ ]:
import nltk
from nltk.corpus import words
from collections import Counter

TXT_PATH = './assets/chat.txt'

nltk.download('words')
spanish_words = set(words.words())

with open(TXT_PATH, 'r', encoding='utf-8') as file:
    lines = file.readlines()

filtered_content = [line.split(':', 1)[-1].strip() for line in lines]
all_words = ' '.join(filtered_content).split()
all_words_lower = [word.lower() for word in all_words]


filtered_words = [word for word in all_words_lower if word in spanish_words]

word_count = Counter(filtered_words)
sorted_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

for word, frequency in sorted_words:
    print(f"{word}: {frequency}")

### Algoritmo
Extensiones del algoritmo original necesarias para la resolución del problema: tratamiento de atributos faltantes, numéricos, etc. (si es el propio algoritmo el que lo maneja), implementaciones adicionales necesarias para manejar ensambles de clasificadores, etc.

### Evaluación
- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.

## Experimentación

### Simulador de cliente.

El experimento consiste en evaluar los distintos hiperparametros del modelo. Alterando el tamaño de la ventana de palabras, y el vocabulario buscamos encontrar un equilibrio entre las siguientes propiedades deseables:

- **Memoria**: El modelo debe ser capaz de sugerir palabras que tengan coherencia con el grupo de Whatsapp de donde se extrajeron los datos.

- **Adaptacion**: El modelo debe ser capaz de adaptarse a la nueva evidencia, y mejorar su performance, sugiriendo palabras que tengan coherencia con la sesion de chat que se esta simulando.

El siguiente script permite simular el comportamiento de un cliente que escribe frases, fue proporcionado por el cuerpo docente. Es de utilidad para evaluar el desempeño del modelo, ya que permite ingresar frases y ver las sugerencias que el modelo realiza.


In [ ]:
def recomendacion_bayesiana(frase):
  import random

  dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

  ## PSEUDOCODIGO - pag 55 - diapo##

  D = ["mi", "hijo", "se", "olvidó", "de", "la"]
  Horizonte = 4 #hiperparametro
  h_MAP = ""
  p_MAP = 0
  for h in P:
    
    prob = P[h]
    
    for d in D[-Horizonte:]:
      prob = prob * PD[h].get(d, P_nada) 
      #P_nada es un valor que debemos definir para el caso cuando la palabra no se encuentre en el listado
    
    if prob > p_MAP:
      h_MAP , p_MAP = h , prob
  print(h_MAP)

  return(random.choice(dias))


##### LOOP PRINCIPAL #####

print("Ingrese la frase dando ENTER luego de \x1b[3mcada palabra\x1b[0m.")
print("Ingrese sólo ENTER para aceptar la recomendación sugerida")
print("Ingrese '.' para comenzar con una frase nueva.")
print("Ingrese '..' para terminar el proceso.")

frase = []
palabra_sugerida = ""

while 1:
  palabra = input(">> ")

  if palabra == "..":
    break

  elif palabra == ".":
    print("----- Comenzando frase nueva -----")
    frase = []

  elif palabra == "": # acepta última palabra sugerida
    frase.append(palabra_sugerida)

  else: # escribió una palabra
    frase.append(palabra)

  if frase:
    palabra_sugerida = recomendacion_bayesiana(frase)

    frase_propuesta = frase.copy()
    frase_propuesta.append("\x1b[3m"+ palabra_sugerida +"\x1b[0m")

    print(" ".join(frase_propuesta))

## Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?